In [7]:
import os
import time

from functions.input2dataframe import *

from functions.analyse_df_pcap import *
from functions.tcpdump_filter_anon import *

from functions.analyse_df_nfdump import *
from functions.nfdump_filter_anon import *

import warnings
warnings.filterwarnings('ignore')

In [8]:
def ddos_analysis_fingerprint_filter_sanitization(input_file, debug=True):
    
    start = time.time()

    if debug: print 'Converting input file to dataframe (time:',  
    df = input2dataframe(input_file) #input2dataframe checks the input_file type (pcap, pcapng, nfdump) and calls the right converter (e.g. pcap2dataframe, nfdump2dataframe)
    if debug: print str(time.time() - start)+')'
    
    #########################################################        
    #########################################################
    start = time.time()
    if debug: print 'Analysing the dataframe for finding attack patterns...'    
        
    if 'tuple' in str(type(df)):
        all_patterns = analyse_df_nfdump(df, debug)
        if debug: print 'Dataframe completely analysed and all attack patterns identified! (time:'+str(time.time() - start)+')'
        #########################################################        
        #########################################################
        start = time.time()
        if debug: print 'Generating attack vectors (with the same format as the input file) and anonymizing the victim information (dst_ip) (time:' 
        for pattern in all_patterns["patterns"]:
            nfdump_filter_anon(input_file,  pattern, all_patterns['dst_ip'])
        #########################################################        
        #########################################################   
    else:
        all_patterns = analyse_df_pcap(df, debug=True)
        display(all_patterns)
        
        if debug: print 'Dataframe completely analysed and all attack patterns identified! (time:'+str(time.time() - start)+')'
        #########################################################        
        #########################################################
        start = time.time()
        if debug: print 'Generating attack vectors (same format as input file) and anonymizing victim\'s information (dst_ip) (time:' 
        for pattern in all_patterns["patterns"]:
            tcpdump_filter_anon(input_file, pattern, all_patterns["dst_ip"])
            
    if debug: print str(time.time() - start)+')'
#     #########################################################        
#     #########################################################
#     #TODO: Generate rule/signatures for specific detection and mitigation technologies (SIDS: BRO, SNORT, SURICATA; Firewall)
    

## Execution from Jupyter Notebook

In [9]:
input_file="input4test/prod-flow-anon-008.nfdump"
ddos_analysis_fingerprint_filter_sanitization(input_file)

Converting input file to dataframe (time: 7.76734209061)
Analysing the dataframe for finding attack patterns...
Total number packets: 457123000

###################################
IDENTIFYING MAIN CHARACTERISTICS:
###################################
Target (destination) IP: 150.61.225.0
Number of packets: 457123000.0

###################################################################################################################
IP protocol used in packets going to target IP: UDP
Number of packets: 336054000.0

#############################
PORT FREQUENCY OF REMAINING PACKETS
##############################
SOURCE ports frequency
src_port
123      32.329328
0         0.230023
19        0.175269
55092     0.003868
19706     0.003868
Name: i_packets, dtype: float64

DESTINATION ports frequency
dst_port
22       49.722068
80       17.482309
10929     1.259917
31765     1.259321
48960     1.252477
Name: i_packets, dtype: float64

 Using top dest port:  22

############################
P

## Main for python script 

In [ ]:
if __name__ == '__main__':
    import argparse
    import os.path
    
    parser = argparse.ArgumentParser(description='')
    
    parser.add_argument('--input', metavar='input_file', required=True,
                        help='Path of a input file')
    args = parser.parse_args()
    
    input_file=args.input
    
    if os.path.isfile(input_file) == False:
        print 'We were unable to find the file. Please check the file path!!'
    
        ddos_analysis_fingerprint_filter_sanitization(input_file, debug=True)